<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=5a979d00ec6a82e5b9434c3a11aeb60b58a49a8331d75146744db99b972e949f
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-07 10:21:41
-------------------
qualified stocks: 87
with latest results: 26
still star stocks: 16
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  1.05 L
Current:  1.42 C
-------------------
Today PnL: -1.49 L (-1.04%)
Current PnL: -23.03 L (-15.09%)
CY Booked + Current PnL: -10.38 L (-6.8%)
-------------------
Total profit:  1.64 L
Total loss:  -24.68 L
-------------------
Total Booked + Current PnL: 16.60 L (13.21%)
Total Booked PnL: 39.63 L (31.55%)
Curr Year Booked PnL: 12.65 L (8.94%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.32 C
Est FTT PnL: 90.02 L (63.6%)
Deployed:  1.26 C
Current:  1.42 C
CAGR/XIRR %: 6.97%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
58,RELIANCE,1533.00,-6.91,67.0,X-LC,6.47,232955.0,22589.0,6197.0,-0.19,10.74,2.66,13.68,37.0,3.65,1.66,29.04,XY25,NTT,REFINERIES
78,TTKPRESTIG,770.00,101.07,51.0,M-SC,1.92,86650.0,-14127.0,14219.0,-0.29,-14.02,16.41,0.09,245.0,-0.99,0.62,13.84,OX40N,NTT,DURABLES
79,UNIONBANK,163.00,-6.98,61.0,M-LC,10.58,169129.0,28289.0,16203.0,-1.84,20.09,9.58,31.59,66.0,1.75,1.20,51.89,XY24,NTT,BANKS
2,ABBOTINDIA,35195.00,-10.86,40.0,X-MC,2.82,88035.0,-2643.0,17554.0,-0.37,-2.91,19.94,16.44,101.0,-0.15,0.63,16.67,X40,ATH,PHARMA
33,ICICIGI,2252.93,-14.54,53.0,X-MC,5.78,146890.0,10757.0,19830.0,-0.80,7.90,13.50,22.47,91.0,0.54,1.05,21.91,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,VOLTAS,1530.00,-6.53,32.0,X-MC,4.35,196050.0,4308.0,33446.0,1.13,2.25,17.06,19.69,99.0,0.13,1.40,9.20,XY25,NTT,AC
37,INDIGOPNTS,1408.00,89.17,36.0,M-SC,6.39,124118.0,-50441.0,50479.0,1.02,-28.90,40.67,0.02,221.0,-1.00,0.88,7.96,OX40N,NTT,PAINTS
82,VALIANTORG,838.00,-284.03,41.0,H-SC,11.30,102340.0,-61265.0,182585.0,0.99,-37.45,178.41,74.15,139.0,-0.34,0.73,31.18,XR,NTT,CHEMICALS
35,IEX,219.00,-33.86,46.0,H-SC,16.01,189302.0,-10652.0,107883.0,0.98,-5.33,56.99,48.63,115.0,-0.10,1.35,5.61,XR,NTT,MISC
14,BSOFT,836.99,-15.18,56.0,H-SC,10.81,95523.0,-45130.0,117914.0,0.44,-32.09,123.44,51.74,131.0,-0.38,0.68,10.52,XR,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
70,SURYODAY,240.0,53.51,41.0,H-SC,15.66,141698.0,-37373.0,103099.0,-6.85,-20.87,72.76,36.71,135.0,-0.36,1.01,40.59,XR,NTT,BANKS
71,SYMPHONY,1306.0,-45.13,41.0,M-SC,16.75,117022.0,-54119.0,54064.0,-4.84,-31.62,46.20,-0.03,196.0,-1.00,0.83,2.02,OX40N,NTT,DURABLES
1,ABB,7934.0,-42.30,36.0,H-LC,5.08,241104.0,-20515.0,139720.0,-3.92,-7.84,57.95,45.57,7.0,-0.15,1.72,2.59,AR,NTT,ELECTRICAL
19,COFFEEDAY,80.0,-51.71,32.0,L-SC,27.46,72864.0,-40685.0,80733.0,-3.21,-35.83,110.80,35.27,268.0,-0.50,0.52,78.34,XR,NTT,HOTELS
17,CERA,9475.0,-24.34,41.0,H-SC,1.85,137724.0,-38179.0,80197.0,-3.18,-21.70,58.23,23.89,149.0,-0.48,0.98,18.94,OX40N,NTT,CERAMICS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
46,KPIGREEN,731.05,5.57,50.0,H-SC,7.93,125685.0,388.0,58544.0,-2.35,0.31,46.58,47.03,141.0,0.01,0.89,57.37,MH,ATH,POWER


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.0,101.07,51.0,M-SC,1.92,86650.0,-14127.0,14219.0,-0.29,-14.02,16.41,0.09,245.0,-0.99,0.62,13.84,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.97,43.0,H-SC,2.74,220977.0,-48690.0,85032.0,-1.07,-18.06,38.48,13.47,138.0,-0.57,1.57,12.36,XY24,NTT,PAINTS
17,CERA,9475.0,-24.34,41.0,H-SC,1.85,137724.0,-38179.0,80197.0,-3.18,-21.70,58.23,23.89,149.0,-0.48,0.98,18.94,OX40N,NTT,CERAMICS
42,JCHAC,2282.0,19921.74,62.0,M-SC,1.50,97832.0,-29973.0,29956.0,-0.06,-23.45,30.62,-0.01,232.0,-1.00,0.70,10.78,OX40N,NTT,AC
67,SIS,528.0,1984.97,41.0,H-SC,2.55,84150.0,-26882.0,50490.0,-2.77,-24.21,60.00,21.26,156.0,-0.53,0.60,13.70,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,UNIONBANK,163.0,-6.98,61.0,M-LC,10.58,169129.0,28289.0,16203.0,-1.84,20.09,9.58,31.59,66.0,1.75,1.2,51.89,XY24,NTT,BANKS


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAIL,228.00,48.36,55.0,M-MC,10.22,234468.0,9506.0,158149.0,-1.27,4.23,67.45,74.53,192.0,0.06,1.67,37.84,XY24,BTT,STEEL
46,KPIGREEN,731.05,5.57,50.0,H-SC,7.93,125685.0,388.0,58544.0,-2.35,0.31,46.58,47.03,141.0,0.01,0.89,57.37,MH,ATH,POWER
79,UNIONBANK,163.00,-6.98,61.0,M-LC,10.58,169129.0,28289.0,16203.0,-1.84,20.09,9.58,31.59,66.0,1.75,1.20,51.89,XY24,NTT,BANKS
85,WHIRLPOOL,2270.00,-41.55,48.0,M-SC,6.14,121155.0,6357.0,87680.0,-2.47,5.54,72.37,81.92,223.0,0.07,0.86,43.43,XR,NTT,DURABLES
36,INDIAMART,4810.62,-53.71,51.0,H-SC,5.40,129325.0,5989.0,125639.0,-1.33,4.86,97.15,106.72,119.0,0.05,0.92,29.37,AR,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WHIRLPOOL,2270.00,-41.55,48.0,M-SC,6.14,121155.0,6357.0,87680.0,-2.47,5.54,72.37,81.92,223.0,0.07,0.86,43.43,XR,NTT,DURABLES
36,INDIAMART,4810.62,-53.71,51.0,H-SC,5.40,129325.0,5989.0,125639.0,-1.33,4.86,97.15,106.72,119.0,0.05,0.92,29.37,AR,ATH,MISC
46,KPIGREEN,731.05,5.57,50.0,H-SC,7.93,125685.0,388.0,58544.0,-2.35,0.31,46.58,47.03,141.0,0.01,0.89,57.37,MH,ATH,POWER
31,HINDZINC,730.22,24.04,42.0,M-LC,9.71,202471.0,-2605.0,115166.0,-0.37,-1.27,56.88,54.89,52.0,-0.02,1.44,22.52,X5K,ATH,METALS
86,WIPRO,420.00,-16.52,38.0,M-LC,5.42,146667.0,-4278.0,113726.0,-1.45,-2.83,77.54,72.51,53.0,-0.04,1.04,3.02,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TATAMOTORS,1065.00,-53.17,22.0,X-LC,32.34,148050.0,-128473.0,238553.0,0.33,-46.46,161.13,39.81,54.0,-0.54,1.05,4.35,XY24,NTT,AUTO
12,BATAINDIA,2096.00,-42.01,23.0,X-SC,7.81,86619.0,-42051.0,87347.0,-0.68,-32.68,100.84,35.20,219.0,-0.48,0.62,0.00,X40,NTT,FOOTWEAR
44,JSWINFRA,342.00,-24.16,26.0,X-MC,4.54,190080.0,-10267.0,40772.0,-0.81,-5.12,21.45,15.23,178.0,-0.25,1.35,25.47,X40N,NTT,REALTY
23,DMART,5391.45,-20.90,26.0,X-LC,7.55,201500.0,-8907.0,68067.0,-1.28,-4.23,33.78,28.12,38.0,-0.13,1.43,18.39,X40N,ATH,FMCG
52,PGHH,17907.65,-32.27,30.0,X-MC,0.15,196020.0,-4800.0,72586.0,-0.71,-2.39,37.03,33.76,80.0,-0.07,1.40,2.68,X40,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,PGHH,17907.65,-32.27,30.0,X-MC,0.15,196020.0,-4800.0,72586.0,-0.71,-2.39,37.03,33.76,80.0,-0.07,1.40,2.68,X40,ATH,FMCG
41,ITC,452.00,-38.34,46.0,X-LC,1.06,199414.0,-724.0,21616.0,0.07,-0.36,10.84,10.44,4.0,-0.03,1.42,5.18,X40,NTT,FMCG
66,SIEMENS,4671.50,-5.61,37.0,H-LC,1.40,151600.0,-34495.0,81970.0,-1.67,-18.54,54.07,25.51,15.0,-0.42,1.08,11.72,AR,ATH,ELECTRICAL
28,HAVELLS,2069.16,-9.44,33.0,X-MC,3.25,302295.0,-25696.0,132224.0,-0.19,-7.83,43.74,32.48,92.0,-0.19,2.15,2.70,X40,ATH,ELECTRICAL
80,UNITDSPR,1644.00,-6.66,64.0,X-LC,4.12,244842.0,10698.0,37926.0,-1.89,4.57,15.49,20.77,86.0,0.28,1.74,11.80,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,2096.00,-42.01,23.0,X-SC,7.81,86619.0,-42051.0,87347.0,-0.68,-32.68,100.84,35.20,219.0,-0.48,0.62,0.00,X40,NTT,FOOTWEAR
53,QUESS,986.00,-50.96,31.0,X-SC,37.46,48570.0,-16436.0,167363.0,-0.60,-25.28,344.58,232.18,198.0,-0.10,0.35,0.00,XY24,NTT,MISC
51,PAGEIND,51769.92,-29.97,32.0,X-MC,7.43,79100.0,-4410.0,24442.0,-0.53,-5.28,30.90,23.98,82.0,-0.18,0.56,0.23,X40,ATH,APPARELS
20,COLPAL,3726.84,-7.40,39.0,X-MC,7.30,216050.0,-47315.0,156636.0,-0.60,-17.97,72.50,41.51,84.0,-0.30,1.54,1.39,XY25,ATH,FMCG
3,ACC,3906.00,-38.93,41.0,X-MC,2.38,182100.0,-55851.0,208504.0,-0.62,-23.47,114.50,64.15,174.0,-0.27,1.30,2.15,XY24,BTT,CEMENT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-50.96,31.0,X-SC,37.46,48570.0,-16436.0,167363.0,-0.60,-25.28,344.58,232.18,198.0,-0.10,0.35,0.00,XY24,NTT,MISC
51,PAGEIND,51769.92,-29.97,32.0,X-MC,7.43,79100.0,-4410.0,24442.0,-0.53,-5.28,30.90,23.98,82.0,-0.18,0.56,0.23,X40,ATH,APPARELS
57,RELAXO,1176.00,-40.85,40.0,X-SC,4.23,80596.0,-64564.0,140495.0,-1.99,-44.48,174.32,52.31,136.0,-0.46,0.57,7.88,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-42.01,23.0,X-SC,7.81,86619.0,-42051.0,87347.0,-0.68,-32.68,100.84,35.20,219.0,-0.48,0.62,0.00,X40,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-10.86,40.0,X-MC,2.82,88035.0,-2643.0,17554.0,-0.37,-2.91,19.94,16.44,101.0,-0.15,0.63,16.67,X40,ATH,PHARMA


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4389.97,-29.97,43.0,X-LC,9.74,279208.0,-66748.0,133461.0,-1.35,-19.29,47.80,19.28,1.0,-0.50,1.99,3.22,X40,ATH,IT
39,INFY,2275.00,-20.32,45.0,X-LC,5.00,309936.0,-2952.0,174649.0,-0.79,-0.94,56.35,54.87,3.0,-0.02,2.21,7.26,X40,BTT,IT
41,ITC,452.00,-38.34,46.0,X-LC,1.06,199414.0,-724.0,21616.0,0.07,-0.36,10.84,10.44,4.0,-0.03,1.42,5.18,X40,NTT,FMCG
83,VBL,671.64,-17.03,52.0,X-LC,6.34,299062.0,-16780.0,129434.0,-0.66,-5.31,43.28,35.67,5.0,-0.13,2.13,7.81,X40N,ATH,FMCG
1,ABB,7934.00,-42.30,36.0,H-LC,5.08,241104.0,-20515.0,139720.0,-3.92,-7.84,57.95,45.57,7.0,-0.15,1.72,2.59,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COFFEEDAY,80.00,-51.71,32.0,L-SC,27.46,72864.0,-40685.0,80733.0,-3.21,-35.83,110.80,35.27,268.0,-0.50,0.52,78.34,XR,NTT,HOTELS
7,ASIANTILES,137.00,7294.44,60.0,L-SC,12.26,81159.0,-12651.0,89267.0,-2.71,-13.49,109.99,81.67,269.0,-0.14,0.58,57.17,XR,NTT,CERAMICS
49,MASFIN,398.61,-20.05,45.0,H-SC,8.31,90600.0,-7380.0,28983.0,-0.72,-7.53,31.99,22.05,152.0,-0.25,0.64,31.69,XR,ATH,FINANCE
85,WHIRLPOOL,2270.00,-41.55,48.0,M-SC,6.14,121155.0,6357.0,87680.0,-2.47,5.54,72.37,81.92,223.0,0.07,0.86,43.43,XR,NTT,DURABLES
46,KPIGREEN,731.05,5.57,50.0,H-SC,7.93,125685.0,388.0,58544.0,-2.35,0.31,46.58,47.03,141.0,0.01,0.89,57.37,MH,ATH,POWER


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,VAIBHAVGBL,521.00,71.90,62.0,H-SC,3.94,148811.0,-33964.0,145031.0,-0.79,-18.58,97.46,60.77,125.0,-0.23,1.06,34.02,XR,NTT,JEWELLERY
68,SONACOMS,806.63,-32.59,62.0,M-SC,6.47,83904.0,-17255.0,57256.0,-0.71,-17.06,68.24,39.54,202.0,-0.30,0.60,18.31,AR,ATH,AUTO
46,KPIGREEN,731.05,5.57,50.0,H-SC,7.93,125685.0,388.0,58544.0,-2.35,0.31,46.58,47.03,141.0,0.01,0.89,57.37,MH,ATH,POWER
62,SAMMAANCAP,326.00,-195.45,66.0,M-SC,33.93,165861.0,16641.0,127547.0,-0.55,11.15,76.90,96.62,208.0,0.13,1.18,79.50,XY25,NTT,FINANCE
7,ASIANTILES,137.00,7294.44,60.0,L-SC,12.26,81159.0,-12651.0,89267.0,-2.71,-13.49,109.99,81.67,269.0,-0.14,0.58,57.17,XR,NTT,CERAMICS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.72
1,25,44.38
2,50,75.47


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.66
LC    33.22
MC    23.10
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.91
X40      16.29
X40N     12.44
XY25     12.01
XR        9.84
AR        7.80
OX40N     6.54
X200      1.78
X5K       1.44
SR        1.04
MH        0.89
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.18
X-LC    22.56
X-MC    18.79
M-SC    12.57
X-SC     5.48
M-LC     5.10
H-LC     4.58
H-MC     2.32
M-MC     1.67
L-SC     1.43
L-LC     0.98
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.37,-4.86,38.76
IT,12.44,-20.26,84.69
FINANCE,12.15,-10.70,60.88
MISC,7.76,-19.32,79.50
ELECTRICAL,5.72,-12.48,52.35
PAINTS,5.61,-16.93,34.37
BANKS,3.99,-20.49,90.58
INSURANCE,3.86,-2.01,40.04
AUTO,2.72,-50.54,113.38


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3371663.0,22
XR,1273639.0,13
AR,1190916.0,9
X40,869903.0,12
X40N,718552.0,9
OX40N,566818.0,9
XY25,487533.0,8
SR,275127.0,2
X5K,115166.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3431085.0,24
M-SC,1279763.0,15
X-MC,1236632.0,14
X-LC,1109326.0,13
X-SC,578738.0,6
H-LC,295593.0,3
M-LC,290444.0,4
H-MC,261781.0,2
L-SC,260726.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1207228.0      6
           AR         844289.0      5
           XR         794636.0      7
M-SC       XY24       741213.0      6
X-MC       XY24       578432.0      4
X-LC       X40        473535.0      5
X-SC       X40N       324028.0      4
X-LC       XY24       317407.0      2
X-MC       X40        309021.0      6
H-SC       SR         275127.0      2
           OX40N      251261.0      3
X-LC       X40N       235427.0      3
H-LC       AR         221690.0      2
X-MC       XY25       190082.0      2
H-MC       XY24       185668.0      1
L-SC       XR         170000.0      2
X-SC       XY24       167363.0      1
X-MC       X40N       159097.0      2
M-MC       XY24       158149.0      1
M-SC       OX40N      148718.0      4
           XR         137348.0      2
           XY25       127547.0      1
           AR         124937.0      2
M-LC       X5K        115166.0      1
           XR         113726.0      1
L-SC       OX40N       90726.0      1
X-SC       X40         87347.0      1
X-LC       XY25        82957.0      3
H-MC       OX40N       76113.0      1
H-LC       X200        73903.0      1
H-SC       MH          58544.0      1
L-MC       XR          57929.0      1
M-LC       XY25        45349.0      1
L-LC       XY25        41598.0      1
M-LC       XY24        16203.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 54.0 seconds
